In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
# instead of using csv file, use pickle to reserve list data type
alcohol_drink = pd.read_pickle('./preprocessed_alcohol.pkl')
alcohol_drink

,DrinkName,ModifiedDate,DrinkID,Alcoholic,Category,ImageUrlPath,Glass,IBA,Instruction,Ingredients,Measures
0,'57 Chevy with a White License Plate,2016-07-18 22:49:04,14029,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,Highball glass,NaN,1. Fill a rocks glass with ice 2.add white cre...,"[[Creme de Cacao, 1], [Vodka, 2]]","[[1 oz white , 1], [1 oz , 2]]"
1,1-900-FUK-MEUP,2016-07-18 22:27:04,15395,Alcoholic,Shot,http://www.thecocktaildb.com/images/media/drin...,Old-fashioned glass,NaN,Shake ingredients in a mixing tin filled with ...,"[[Absolut Kurant, 1], [Grand Marnier, 2], [Cha...","[[1/2 oz , 1], [1/4 oz , 2], [1/4 oz , 3], [1/..."
2,110 in the shade,2016-02-03 14:51:57,15423,Alcoholic,Beer,http://www.thecocktaildb.com/images/media/drin...,Beer Glass,NaN,Drop shooter in glass. Fill with beer,"[[Lager, 1], [Tequila, 2]]","[[16 oz , 1], [1.5 oz , 2]]"
3,151 Florida Bushwacker,2016-07-18 22:28:43,14588,Alcoholic,Milk / Float / Shake,http://www.thecocktaildb.com/images/media/drin...,Beer mug,NaN,Combine all ingredients. Blend until smooth. G...,"[[Malibu rum, 1], [Light rum, 2], [151 proof r...","[[1/2 oz , 1], [1/2 oz , 2], [1/2 oz Bacardi ,..."
4,155 Belmont,2016-10-05 12:36:28,15346,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,White wine glass,NaN,Blend with ice. Serve in a wine glass. Garnish...,"[[Dark rum, 1], [Light rum, 2], [Vodka, 3], [O...","[[1 shot , 1], [2 shots , 2], [1 shot , 3], [1..."
...,...,...,...,...,...,...,...,...,...,...,...
541,Zippy's Revenge,2017-01-28 16:18:33,14065,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,Old-fashioned glass,NaN,Mix Kool-Aid to taste then add Rum and ammaret...,"[[Amaretto, 1], [Rum, 2], [Kool-Aid, 3]]","[[2 oz , 1], [2 oz , 2], [4 oz Grape , 3]]"
542,Zizi Coin-coin,2017-01-28 16:25:52,14594,Alcoholic,Punch / Party Drink,http://www.thecocktaildb.com/images/media/drin...,Margarita/Coupette glass,NaN,"Pour 5cl of Cointreau on ice, add 2cl of fresh...","[[Cointreau, 1], [Lemon juice, 2], [Ice, 3], [...","[[5 cl , 1], [2 cl , 2], [ cubes\n, 3], [ or l..."
543,Zoksel,2017-01-28 16:28:50,15691,Alcoholic,Soft Drink / Soda,http://www.thecocktaildb.com/images/media/drin...,Beer pilsner,NaN,"No specific mixinginstructions, just poor ever...","[[Beer, 1], [Root beer, 2], [Lemonade, 3], [Co...","[[ slice\n, 4]]"
544,Zorbatini,2017-01-28 16:32:35,16963,Alcoholic,Cocktail,http://www.thecocktaildb.com/images/media/drin...,Cocktail glass,NaN,Prepare like a Martini. Garnish with a green o...,"[[Vodka, 1], [Ouzo, 2]]","[[1 1/4 oz Stoli , 1], [1/4 oz , 2]]"


#### preprocess instruction

remove \r, \n in the text

In [3]:
text = 'Place sugar cube in old fashioned glass and saturate with bitters, add a dash of plain water. Muddle until dissolved.\r\nFill the glass with ice cubes and add whiskey.\r\n\r\nGarnish with orange twist, and a cocktail cherry.'
text = re.sub('\\r\\n', '', text)
print(text)

Place sugar cube in old fashioned glass and saturate with bitters, add a dash of plain water. Muddle until dissolved.Fill the glass with ice cubes and add whiskey.Garnish with orange twist, and a cocktail cherry.


In [4]:
def preprocess_instruction(s):
    s = str(s)
    s = re.sub('\\r\\n', '', s)
    return s
alcohol_drink['Instruction'] = alcohol_drink['Instruction'].apply(preprocess_instruction)

#### Analysis Ingredients and Measures

Ingredients와 Measures를 서로 매칭시킬 필요가 있습니다.

1. 때때로, Ingredients와 Measure의 갯수가 서로 다르게 나타나며, 대게 Ingredients의 갯수가 Measure보다
많지만 가끔 Measure의 수가 더 많을 때가 있습니다.

2. 또한 Measure 데이터는 지저분한 편으로, 공백문자나 반점 등이 많이 포함되어 있습니다.

3. 몇몇 Measure값은 빈 리스트([]) 입니다.

4. 단위 뿐만 아니라 일부 재료를 나타내는 단어가 포함되어 있는 경우도 있습니다.

5. 숫자 앞에 숫자가 붙은 경우 해석이 애매합니다. (ex 1 1/2 oz)

6. 어떤한 단위가 사용되었는지 기록하고, 단위와 숫자를 제외한 문자를 제거하여 정제하도록 합니다.

7. 단위가 생략되었던지, 수량이 생략된 경우도 있습니다.

In [5]:
ingredient_list = alcohol_drink['Ingredients'].to_list()
measure_list = alcohol_drink['Measures'].to_list()

uniq_ingr = np.unique(ingredient_list)
uniq_meas = np.unique(measure_list)


/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


많은 행들이 서로 재료와 측정량 갯수가 일치하지 않습니다.

하지만 측정값이 없는 경우(빈 리스트인 경우), 사이트를 참조해 보면 instruction에 정보가 나와있는 경우도 있습니다.

그러나 instruction에서도 정보가 없는 경우도 있습니다.

측정값 중에서 \r\n으로 노이즈가 들어가 있는 경우도 있습니다. 이런 경우 제거가 필요합니다.

In [6]:
# check if the length of measure and ingredient is different
for i in range(len(alcohol_drink)):
    meas_item = alcohol_drink["Measures"][i]
    ingr_item = alcohol_drink["Ingredients"][i]
    # if (len(ingr_item) != len(meas_item)):
    print("Index : ", i)
    print(alcohol_drink['DrinkName'][i])
    print(ingr_item, len(ingr_item))
    print(meas_item, len(meas_item))
    print()

# In case of empty measure list
# 501 Blue, After Five, Amaretto Sweet & Sour ... -> https://www.thecocktaildb.com/
# there is no any usage measurement on the site.. but some cases it is specified on the instruction
# this is also same case for the unspecified amounts


Index :  0
'57 Chevy with a White License Plate
[['Creme de Cacao', 1], ['Vodka', 2]] 2
[['1 oz white ', 1], ['1 oz ', 2]] 2

Index :  1
1-900-FUK-MEUP
[['Absolut Kurant', 1], ['Grand Marnier', 2], ['Chambord raspberry liqueur', 3], ['Midori melon liqueur', 4], ['Malibu rum', 5], ['Amaretto', 6], ['Cranberry juice', 7], ['Pineapple juice', 8]] 8
[['1/2 oz ', 1], ['1/4 oz ', 2], ['1/4 oz ', 3], ['1/4 oz ', 4], ['1/4 oz ', 5], ['1/4 oz ', 6], ['1/2 oz ', 7], ['1/4 oz ', 8]] 8

Index :  2
110 in the shade
[['Lager', 1], ['Tequila', 2]] 2
[['16 oz ', 1], ['1.5 oz ', 2]] 2

Index :  3
151 Florida Bushwacker
[['Malibu rum', 1], ['Light rum', 2], ['151 proof rum', 3], ['Dark Creme de Cacao', 4], ['Cointreau', 5], ['Milk', 6], ['Coconut liqueur', 7], ['Vanilla ice-cream', 8]] 8
[['1/2 oz ', 1], ['1/2 oz ', 2], ['1/2 oz Bacardi ', 3], ['1 oz ', 4], ['1 oz ', 5], ['3 oz ', 6], ['1 oz ', 7], ['1 cup ', 8]] 8

Index :  4
155 Belmont
[['Dark rum', 1], ['Light rum', 2], ['Vodka', 3], ['Orange juice'

In [7]:
# 측정 단위 # -> 해당 단위는 제외하고 전처리 하기를 바랍니다.

# Oz -> Ounce -> 28.35g(in case of weight) / 28.41ml (in case of volumn)
# fl. Oz. -> 28.41ml (Volumn metric)
# cl -> Centilitre -> 0.01L -> 10ml (Volumn metric)
# tsp -> teaspoon -> 5.69grams (depend on ingredient) / 4.92ml
# cubes -> ice cube
# tblsp -> 14.78ml
# dash -> 0.92ml
# cup -> 236.588ml
# pinch -> 0.3ml
# shot -> 44.36ml
# glasses -> 200-250ml
# drop -> 0.05ml (한 방울...)
# can -> 355ml (standard)
# jigger -> 44.36ml
# dl -> 100ml (deci liter)
# bottle -> about one bottle... (standard 750ml..)
# pint -> 473.176ml
# quart, qt -> 946.353ml
# L -> 1000ml
# gr -> gram
# inch -> 16.38ml
# splash -> 5.91ml
# gal -> 3785.41ml
# scoop -> 59ml to 32ml (#16 to #30)
# kg -> 1000g
# lb -> pounds -> 0.45kg -> 450g
# ml -> 1ml
# dash of -> 0.92 ml


# part(s) -> ratio of some unit (ounce, jiggers ...)
# top up with -> full glasses with something..
# fill to top -> 가득 채우다
# fill with -> 채우다
# to fill -> 채우다
# fill -> 채우다
# A little bit of -> 약간
# slice -> depend on ingredients
# juice of -> depend on the fruit (etc lemon->about 45ml, orange...)
# sprig -> 2-4 inch piece of herb
# wedge -> thin part of something.
# twist of -> used with lemon peel... just circled lemon peel
# long strip -> used with lemon peel...
# whole, large -> unit quantity
# fifth -> 1/5
# piece -> 1 piece sugar..
# chunks -> depend on fruit
# By taste -> depend on your taste
# to taste -> depend on your taste
# Handful -> 한줌량 (Nut기준 36.3g)
# stick -> 스틱
# packages -> 패키지용량에 따라..
# leaves -> 잎사귀

# Coarse는 단위가 아니다. Coarse Salt는 굵은 소금을 의미하는에 여기서 잘린것 같다..


In [8]:
# let's check what some value is missing....
print(uniq_meas)

[list([]) list([[' cubes\n', 1]]) list([[' slice\n', 4]])
 list([[', orange\n', 2]]) list([['0.75 oz ', 1], ['0.25 oz ', 2]])
 list([['1 ', 1], ['1 shot ', 2]]) list([['1 ', 1], ['2 ', 2]])
 list([['1 1/2 ', 1], ['1/3 oz cream ', 2], ['Juice of 1/2 ', 3]])
 list([['1 1/2 L ', 1], ['1 cup ', 2], ['1 large ', 3], ['1 large ', 4], ['1 large ', 5], ['3-4 oz plain ', 6]])
 list([['1 1/2 cl ', 1]])
 list([['1 1/2 cl ', 1], ['1 1/2 cl ', 2], ['1 1/2 cl ', 3], ['3 cl ', 4]])
 list([['1 1/2 cup ', 1], ['4 tsp ', 2], ['1/2 cup ', 3]])
 list([['1 1/2 oz ', 1], ['1 ', 2]])
 list([['1 1/2 oz ', 1], ['1 1/2 oz ', 2]])
 list([['1 1/2 oz ', 1], ['1 1/2 oz ', 2], ['1 1/4 tsp ', 3], ['1 twist of ', 4]])
 list([['1 1/2 oz ', 1], ['1 1/2 oz ', 2], ['1/4 tsp ', 3], ['1/4 tsp ', 4], ['2 dashes ', 5], ['1 ', 6]])
 list([['1 1/2 oz ', 1], ['1 1/2 tsp ', 2], ['1 tblsp ', 3], ['1 tsp ', 4], ['1 ', 5], ['1 ', 6]])
 list([['1 1/2 oz ', 1], ['1 dash ', 2], ['1 tblsp ', 3]])
 list([['1 1/2 oz ', 1], ['1 dash ', 2],

In [9]:
length_list  = []
for i in range(len(alcohol_drink)):
    length_list.append(len(alcohol_drink['Measures'][i]))

In [10]:
def remove_rn_measure(list):
    if(len(list)!=0):
        num_item = len(list)
        index_list = []
        for i in range(num_item):
            if list[i][0] == '\r\n':
                index_list.append(i)
            list[i][0] = re.sub('\\r|\\n', '', list[i][0])
        for index in reversed(index_list):
            list.pop(index)
        
        return list
    else:
        return list

alcohol_drink['Measures'] = alcohol_drink['Measures'].apply(remove_rn_measure)

In [11]:
# length reduced..
for i in range(len(length_list)):
    if (length_list[i] != len(alcohol_drink['Measures'][i])):
        print(length_list[i], len(alcohol_drink['Measures'][i]))


7 4
7 4
7 3
7 5
7 3
7 2
7 4
7 5


In [12]:
def convert_to_lower_case(list):
    if len(list) != 0:
        for i in range(len(list)):
            list[i][0] = list[i][0].lower()
    return list
alcohol_drink['Measures'] = alcohol_drink['Measures'].apply(convert_to_lower_case)


In [13]:
# make filter that extract unit and number
UNIT = ["oz", "ounce", "fl oz", "cl", "tsp", "cubes", "tblsp", "tablespoon", "dash", "dashes", "cup", "pinch", "shot", "glass", "glasses", "drop",
        "can", "jigger", "dl", "bottle", "pint", "quart", "qt", "gr", "inch", "splash", "gal", "scoop", "scoops",
        "kg", "lb", "ml", "part", "top up with", "fill to top", "fill", "to fill", "a little bit of", "slice", "juice of",
        "sprig", "sprigs", "wedge", "twist of", "long strip", "whole", "large", "fifth", "piece", "pieces", "chunk", "by taste", "to taste",
        "handful", "stick", "package", "packages", "leaves", "l", "dash of"]

filter = ""
for unit in UNIT:
    if unit == "oz":
        filter += unit
        continue
    filter += "|"+unit

print(filter)


oz|ounce|fl oz|cl|tsp|cubes|tblsp|tablespoon|dash|dashes|cup|pinch|shot|glass|glasses|drop|can|jigger|dl|bottle|pint|quart|qt|gr|inch|splash|gal|scoop|scoops|kg|lb|ml|part|top up with|fill to top|fill|to fill|a little bit of|slice|juice of|sprig|sprigs|wedge|twist of|long strip|whole|large|fifth|piece|pieces|chunk|by taste|to taste|handful|stick|package|packages|leaves|l


In [14]:
# regex example

text = " 1 1/2 oz 1.2 top up with 70ml/2fl oz white gin"
result = re.findall(filter+"|\d+[/.]{0,2}\d*|\d+", text)

text2 = "full glasses"
result2 = re.findall(filter+"|\d+[/.]{0,2}\d*|\d+", text2)

text3 = "small bottle"
result3 = re.findall(filter+"|\d+[/.]{0,2}\d*|\d+", text3)

text4 = "3 2-inch strips"
result4 = re.findall(filter+"|\d+[/.]{0,2}\d*|\d+", text4)

text5 = "70ml/2fl oz"
result5 = re.findall(filter+"|\d+[/.]{0,2}\d*|\d+", text5)

print(result)
print(result2)
print(result3)
print(result4)
print(result5)

['1', '1/2', 'oz', '1.2', 'top up with', '70', 'ml', '2', 'fl oz']
['l', 'l', 'glass']
['l', 'l', 'bottle']
['3', '2', 'inch']
['70', 'ml', '2', 'fl oz']


In [15]:
UNIT_NUM = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

def remove_comma_non_unit(list):
    if len(list) != 0:
        length = len(list)
        for index in range(length):
            list[index][0] = str(list[index][0])
            list[index][0] = re.sub(',', '', list[index][0])
            filtered = re.findall(filter+"|\d+[/.-]{0,2}\d*|\d+", list[index][0])
            list[index][0] = filtered
    return list

alcohol_drink['Measures'] = alcohol_drink['Measures'].apply(remove_comma_non_unit)
# test = alcohol_drink['Measures'].apply(remove_comma_non_unit)

In [16]:
# sorted(test)
sorted(alcohol_drink['Measures']) # 1 1/2 -> 1.5와 같음


[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [[[], 1],
  [['2', 'tsp'], 2],
  [['4', 'cl'], 3],
  [['top up with'], 4],
  [['fill'], 5]],
 [[[], 2]],
 [[[], 2]],
 [[['0.75', 'oz'], 1], [['0.25', 'oz'], 2]],
 [[['1'], 1], [['1', 'shot'], 2]],
 [[['1'], 1], [['2'], 2]],
 [[['1', '1/2'], 1], [['1/3', 'oz'], 2], [['juice of', '1/2'], 3]],
 [[['1', '1/2', 'cl'], 1]],
 [[['1', '1/2', 'cl'], 1],
  [['1', '1/2', 'cl'], 2],
  [['1', '1/2', 'cl'], 3],
  [['3', 'cl'], 4]],
 [[['1', '1/2', 'cup'], 1], [['4', 'tsp'], 2], [['1/2', 'cup'], 3]],
 [[['1', '1/2', 'l'], 1],
  [['1', 'cup'], 2],
  [['1', 'large'], 3],
  [['1', 'large'], 4],
  [['1', 'large'], 5],
  [['3-4', 'oz', 'l'], 6]],
 [[['1', '1/2', 'oz'], 1], [['1'], 2]],
 [[['1', '1/2', 'oz'], 1], [['1', '1/2', 'oz'], 2]],
 [[['1', '1/2', 'oz'], 1], [['1', '1/2', 'oz'], 2]],
 [[['1', '1/2', 'oz'], 1],
  [['1', '1/2', 'oz'], 2],
  [['1', '1/4', 'tsp'], 3],
  [['1', 'twist of'], 4]],
 [[['1', '1/2', 'oz'], 1],
  [['1', '1/2', 'oz'], 2],
  [['1/4', 'tsp'], 3

#### Ingredients preprocessing

1. Ingredient 내의 문자들을 모두 소문자로 바꿉니다..

In [17]:
alcohol_drink['Ingredients']

0                      [[Creme de Cacao, 1], [Vodka, 2]]
1      [[Absolut Kurant, 1], [Grand Marnier, 2], [Cha...
2                             [[Lager, 1], [Tequila, 2]]
3      [[Malibu rum, 1], [Light rum, 2], [151 proof r...
4      [[Dark rum, 1], [Light rum, 2], [Vodka, 3], [O...
                             ...                        
541             [[Amaretto, 1], [Rum, 2], [Kool-Aid, 3]]
542    [[Cointreau, 1], [Lemon juice, 2], [Ice, 3], [...
543    [[Beer, 1], [Root beer, 2], [Lemonade, 3], [Co...
544                              [[Vodka, 1], [Ouzo, 2]]
545    [[Sambuca, 1], [Bailey's irish cream, 2], [Whi...
Name: Ingredients, Length: 546, dtype: object

In [18]:
def to_lower_case(list):
    for i in range(len(list)):
        list[i][0] = list[i][0].lower()
    return list
alcohol_drink['Ingredients'] = alcohol_drink['Ingredients'].apply(to_lower_case)

In [19]:
sorted(alcohol_drink['Ingredients'])

[[['151 proof rum', 1], ['kool-aid', 2]],
 [['151 proof rum', 1],
  ['malibu rum', 2],
  ['pineapple juice', 3],
  ['orange juice', 4],
  ['grenadine', 5],
  ['cherries', 6]],
 [['151 proof rum', 1], ['pina colada mix', 2], ['daiquiri mix', 3]],
 [['151 proof rum', 1], ['wild turkey', 2]],
 [['151 proof rum', 1], ['wild turkey', 2]],
 [['7-up', 1], ['apple schnapps', 2]],
 [['absinthe', 1], ['tequila', 2], ['grenadine', 3]],
 [['absolut citron', 1],
  ['grand marnier', 2],
  ['lemon juice', 3],
  ['club soda', 4]],
 [['absolut citron', 1],
  ['lime juice', 2],
  ['cointreau', 3],
  ['cranberry juice', 4]],
 [['absolut citron', 1], ['lime juice', 2], ['ice', 3], ['tonic water', 4]],
 [['absolut citron', 1],
  ['orange curacao', 2],
  ['strawberry liqueur', 3],
  ['orange juice', 4]],
 [['absolut citron', 1],
  ['orange juice', 2],
  ['triple sec', 3],
  ['ginger ale', 4]],
 [['absolut citron', 1], ['pisang ambon', 2], ['ice', 3], ['bitter lemon', 4]],
 [['absolut citron', 1],
  ['sweet 

In [20]:
alcohol_drink.to_pickle('./ing_meas_processed_alcohol.pkl')

In [21]:
test = pd.read_pickle('./ing_meas_processed_alcohol.pkl')

#### 앞으로 추가적으로 할 수 있는 작업

Measure에 빈 값이나, 값이 누락된 부분을 찾아서 채울 수 있습니다.